        Hola Mariano, no se porque la matriz reducida me queda singular en el ejercicio del puente. Primero hice el codigo para resolver el ejemplo de la mensula, con ese ejercicio me da bien, pero despues cuando le cargo los datos del puente me dice que es singular, no entiendo porque.

Te de be faltar alguna condición de contorno.

            Tambien tengo un par de dudas sobre como hice el codigo:
            Primero respecto de como calcular los angulos, para los primeros elementos hago la resta entre un elemento y el siguiente, pero para el ultimo, como le quiero restar el primero, lo tengo que hacer aparte, por eso hago un if para los primeros elementos y otro if para el ultimo, pero seguramente hay una mejor manera de hacerlo.

lo que pasa es que estabas recorriendo por nodo en vez de elemento, con lo cual te quedaban mal definidos los angulos.

            Segundo, para hacer que los valores muy chicos en vez de mostrarse E-33 se muestre como 0, intente usar la forma que vimos en clase pero no se si la escribi mal o que, no me funciono, asi que hice un doble for pero bueno es muy ineficiente. Seria esta parte, igual esta comentada en el codigo:
            #    scale = np.max(np.max(MR_global))
            #    MR_global[abs(MR_global[i][j]/scale) < 1e-9] = 0
            
la linea es:
```.python
    MR_global[abs(MR_global/scale) < 1e-9] = 0
```


In [6]:
import numpy as np
import pdb

In [45]:
def calculos(MN, MC, K, s, r, U, F):

    n = len(MN) #numero de nodos
    gl = 2 # grados de libertad
    MR_global = np.zeros([gl*n,gl*n])
    # un 0 en la matriz F significa que al moverse en esa direccion no hay resistencia
    #MDF_COMMENT te falta el numero de elementos:
    NE = len(MC)

# los elementos estan en la matriz de conectividad:
# MDF_CONNECT    for e in range(n):
    for e in range(NE):
        
        # este if es para el ultimo elemento, le resta al ultimo elemento el primer elemento
#MDF_COMMENT        if e==(n-1):
#MDF_COMMENT            delta_x = MN[0][0] - MN[e][0]
#MDF_COMMENT            delta_y = MN[0][1] - MN[e][1]
#MDF_COMMENT            angulo = np.arctan2(delta_x, delta_y) # para ver en que cuadrante estoy y no tener problemas de signo
#MDF_COMMENT            C = np.cos(angulo)
#MDF_COMMENT            S = np.sin(angulo)
#MDF_COMMENT            MR_local = np.array([(C**2,C*S,-C**2,-C*S),(C*S,S**2,-C*S,-S**2),(-C**2,-C*S,C**2,C*S),(-C*S,-S**2, C*S,S**2)])
#MDF_COMMENT            for i in MC[e]:
#MDF_COMMENT                for j in MC[e]:
#MDF_COMMENT                    columna_i = list(MC[e]).index(i) # identifico en que columna de MC esta el elemento i
#MDF_COMMENT                    columna_j = list(MC[e]).index(j) # # identifico en que columna de MC esta el elemento i
#MDF_COMMENT                    MR_global[i*gl:(i+1)*gl , j*gl:(j+1)*gl] += K[e] * (MR_local[columna_i*gl:(columna_i+1)*gl , columna_j*gl:(columna_j+1)*gl])


#MDF_COMMENT        else:
#MDF_COMMENT los elementos  que tenes que considerar estan en la matriz de conectividad. 
        Nodo_1 = MC[e,0]
        Nodo_2 = MC[e,1]
#MDF_COMMENT        delta_x = MN[e][0] - MN[e][0]
#MDF_COMMENT        delta_y = MN[e][1] - MN[e][1]
        delta_x = MN[Nodo_1][0] - MN[Nodo_2][0]
        delta_y = MN[Nodo_1][1] - MN[Nodo_2][1]
        #MDF_COMMENT te faltan las longitudes:
        L = np.sqrt(delta_x**2 + delta_y**2)
        angulo = np.arctan2(delta_y, delta_x) # para ver en que cuadrante estoy y no tener problemas de signo
        C = np.cos(angulo)
        S = np.sin(angulo)
        MR_local =( K[e] / L ) * np.array(
            [(C**2,C*S,-C**2,-C*S),
             (C*S,S**2,-C*S,-S**2),
             (-C**2,-C*S,C**2,C*S),
             (-C*S,-S**2, C*S,S**2)])

        # esta parte esta eimpecable. ahora vemos que pasa cuando la corremos. 
        # con esa definición i es el numero de nodo !
        for i in MC[e]:
            # con esa definición columna_i es el indice local !
            columna_i = list(MC[e]).index(i) # identifico en que columna de MC esta el elemento i
            #MDF_COMMENT para que te de todos los elementos de matriz que necesitas tenes que 
            #MDF_COMMENT empezar asi:
            index_i = np.linspace(i*gl,(i+1)*gl-1,gl).astype(int)
            ind_col_i = np.linspace(columna_i*gl,(columna_i+1)*gl-1,gl).astype(int)
            for j in MC[e]:
                columna_j = list(MC[e]).index(j) # # identifico en que columna de MC esta el elemento i
                index_j = np.linspace(j*gl,(j+1)*gl-1,gl).astype(int)
                ind_col_j = np.linspace(columna_j*gl,(columna_j+1)*gl-1,gl).astype(int)
                #MDF_COMMENT te falta el truco de np.ix_
                #MDF_COMMENT MR_global[i*gl:(i+1)*gl , j*gl:(j+1)*gl] += \
                #MDF_COMMENT K[e] * MR_local[columna_i*gl:(columna_i+1)*gl , columna_j*gl:(columna_j+1)*gl]
                MR_global[np.ix_(index_i, index_j)] +=  MR_local[np.ix_(ind_col_i , ind_col_j)]
                    

    scale = np.max(np.max(MR_global))
    #            MR_global[abs(MR_global[i][j]/scale) < 1e-9] = 0
    #MDF_COMMENT lo que pasa es que MR_Globlal es np array y no necesitas indexar asi, le mandas toda la matriz y listo:
    MR_global[abs(MR_global/scale) < 1e-9] = 0
    # basicamente abs(MR_global/scale) < 1e-9 te devuelve una matriz de True en los lugares donde
    # se cumple la desigualdad y False donde no. eso como indice the da un slice de MR_Global .
    # esta linea de arriba no me funciona, por eso hago el doble for
#    for i in range(len(MR_global)):
#        for j in range(len(MR_global)):
 #           if abs(MR_global[i][j]) < 1e-9:
 #               MR_global[i][j] = 0


    # determinante = 0, deja de ser 0 cuando se usa la matriz reducida
    MR_reducida = MR_global.copy()
    # borro filas y columnas de las posiciones de U que ya tengo
    MR_reducida = np.delete(MR_reducida, s, axis=0)
    MR_reducida = np.delete(MR_reducida, s, axis=1)

    #MDF_COMMENT es una forma razonable de hacerlo, pero:
    # - estas copiando la matriz. si el sistema es muy grande esto es ineficiente. 
    # - también es mas lento borrar filas que simplemente hacer slice. 
    
    nuevas_U = np.linalg.solve(MR_reducida, F) # obtengo las U que no conozco
    # -justo anoche hablamos que en esta ecuación te falta la parte de los desplazamientos vinculados. 


    # armo el vector U completo
    U_global = np.zeros(n*gl)
    k=0
    z=0
    # k y z son contadores que van recorriendo en orden los vectores U y nuevas_U

    for i in range(len(U_global)):
        if i in s:
            U_global[i] = U[k]
            k += 1

        if i in r:
            U_global[i] = nuevas_U[z]
            z += 1

    # ahora obtengo el vector F completo
    F = np.matmul(MR_global,U_global)

    return(F, U_global, MR_global)


In [41]:
## EJEMPLO MENSULA
MN = np.array([(0,0), (1,0), (1,1)])# matriz_nodos: matriz con las posiciones, en cada fila estan los nodos y cada columna las coordenadas x,y,z, etc
MC = np.array([(0,1), (1,2), (2,0)]) # matriz_conectividad: fila cada element, columna cada nodo que forman ese elemento
#MDF_COMMENT K = [10,5,20] # vector con las constantes elasticas de cada elemento
K = 210E9 / np.array([10e-4, 10e-4,10e-4 ])   # E / A, faltan las longitudes
s = [0,1,3] # posiciones de los desplazamientos que conozco
r = [2,4,5] # posiciones de las fuerzas que conozco
U = [0,0,0] # valores de los desplazamientos, respectivamente a s
F = [0,2,1] # valores de las fuerzas, respectivamente a r

In [46]:
F, U, matriz_rigidez = calculos(MN, MC, K, s, r, U, F)

print('Fuerzas =', F)
print('Desplazamientos =', U)

Fuerzas = [-2. -2.  0.  1.  2.  1.]
Desplazamientos = [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3.16993059e-14 -4.76190476e-15]


In [49]:
# EJERCICIO 2 GUIA
MN = np.array([[0,0] , [800,400] , [1600,0] , [800,0]]) # en mm
MC = np.array([[0,1] , [3,0] , [3,1] , [1,2], [2, 3]])


A = [1000, 1000, 2000, 1000, 1000] # area en mm2
E = 210 * np.ones(len(MC)) # Gpa = N/mm2
l0 = np.sqrt(800**2 + 400**2)
l3 = np.sqrt(800**2 + 400**2)
L = [l0,800,400,l3,800 ]

K = np.zeros(len(MC))
for i in range(len(K)):
    K[i] = (E[i]*A[i])/L[i]

U = [0,0,0]
s = [0,1,5]

F = [0,0,0,0,-20000] # en Newton
r = [2,3,4,6,7]

In [50]:
F, U, matriz_rigidez = calculos(MN, MC, K, s, r, U, F)

print('Fuerzas =', F)
print('Desplazamientos =', U)

Fuerzas = [ 7.27595761e-12  1.00000000e+04  0.00000000e+00  7.27595761e-12
 -7.27595761e-12  1.00000000e+04  0.00000000e+00 -2.00000000e+04]
Desplazamientos = [      0.               0.           60952.38095238 -312380.95238095
  121904.76190476       0.           60952.38095238 -320000.        ]
